# CISE v1 Demo

**Constraint-Induced Structure Explorer**

This notebook demonstrates the core functionality of CISE: generating baseline ensembles, applying constraints via energy-based reweighting, and comparing the resulting distributions.

---

**Disclaimer**: This is a constraint experiment exploring how penalty functions affect ensemble distributions. Results should not be interpreted as physical predictions.

In [ ]:
# Setup
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt

from cise.sampling import GaussianSampler, UniformSampler
from cise.constraints import SmoothnessConstraint, L1Constraint, LowRankConstraint, HierarchyConstraint
from cise.methods.reweighting import apply_reweighting, compute_ess
from cise.analysis.metrics import compute_norm_stats, compute_compressibility
from cise.analysis.embeddings import compute_pca, compute_intrinsic_dimension

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

## 1. Generating Baseline Ensembles

CISE supports two baseline samplers:
- **Gaussian**: Components drawn from N(0, 1)
- **Uniform**: Components drawn from U(-1, 1)

In [ ]:
# Configuration
SEED = 1337
N_SAMPLES = 5000
VECTOR_DIM = 32
MATRIX_SIZE = 6

# Generate vector samples
gaussian_sampler = GaussianSampler(seed=SEED)
vectors = gaussian_sampler.sample_vectors(N_SAMPLES, VECTOR_DIM)

print(f"Generated {N_SAMPLES} vectors of dimension {VECTOR_DIM}")
print(f"Shape: {vectors.shape}")
print(f"Mean: {vectors.mean():.4f}, Std: {vectors.std():.4f}")

In [ ]:
# Generate matrix samples
matrices = gaussian_sampler.sample_matrices(N_SAMPLES, MATRIX_SIZE)

print(f"Generated {N_SAMPLES} matrices of size {MATRIX_SIZE}x{MATRIX_SIZE}")
print(f"Shape: {matrices.shape}")

## 2. Defining Constraints

Constraints are implemented as penalty functions that assign an energy E(z) to each sample.

In [ ]:
# Initialize constraints for vectors
smoothness = SmoothnessConstraint(scale=1.0)
l1_constraint = L1Constraint(scale=0.1)  # Scaled down for balance

# Compute energies
smoothness_energies = smoothness.energy(vectors)
l1_energies = l1_constraint.energy(vectors)

print(f"Smoothness energy: mean={smoothness_energies.mean():.2f}, std={smoothness_energies.std():.2f}")
print(f"L1 energy: mean={l1_energies.mean():.2f}, std={l1_energies.std():.2f}")

In [ ]:
# Plot energy distributions
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(smoothness_energies, bins=50, alpha=0.7, color='steelblue')
axes[0].set_xlabel('Smoothness Energy')
axes[0].set_ylabel('Count')
axes[0].set_title('Smoothness Constraint Energy Distribution')

axes[1].hist(l1_energies, bins=50, alpha=0.7, color='darkorange')
axes[1].set_xlabel('L1 Energy')
axes[1].set_ylabel('Count')
axes[1].set_title('L1 Constraint Energy Distribution')

plt.tight_layout()
plt.show()

## 3. Energy-Based Reweighting

The core of CISE is applying constraints via Boltzmann weights:

$$w(z) = \exp(-\beta \cdot E(z))$$

where $\beta$ controls the constraint strength.

In [ ]:
# Apply reweighting with different beta values
constraints = [smoothness, l1_constraint]
beta_values = [0.0, 0.5, 1.0, 2.0]

results = {}
for beta in beta_values:
    result = apply_reweighting(vectors, constraints, beta=beta, resample=True)
    results[beta] = result
    print(f"Beta={beta}: ESS={result.ess:.1f} ({result.ess_ratio:.1%})")

In [ ]:
# Visualize weight distributions
fig, axes = plt.subplots(1, len(beta_values), figsize=(14, 3))

for ax, beta in zip(axes, beta_values):
    weights = results[beta].weights
    ax.hist(weights, bins=50, alpha=0.7, color='steelblue')
    ax.set_xlabel('Weight')
    ax.set_ylabel('Count')
    ax.set_title(f'β={beta}')
    ax.ticklabel_format(style='scientific', axis='x', scilimits=(0,0))

plt.suptitle('Weight Distributions by Beta Value', fontsize=12)
plt.tight_layout()
plt.show()

## 4. Comparing Baseline vs Constrained Distributions

In [ ]:
# Compare energy distributions
beta = 1.0
result = results[beta]

fig, ax = plt.subplots(figsize=(8, 5))

ax.hist(result.energies, bins=50, alpha=0.6, label='Baseline', density=True, color='steelblue')
ax.hist(result.energies, bins=50, weights=result.weights, alpha=0.6, 
        label=f'Constrained (β={beta})', density=True, color='darkorange')

ax.set_xlabel('Total Energy')
ax.set_ylabel('Density')
ax.set_title('Energy Distribution: Baseline vs Constrained')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Compare norms
norms = np.linalg.norm(vectors, axis=1)

fig, ax = plt.subplots(figsize=(8, 5))

ax.hist(norms, bins=50, alpha=0.6, label='Baseline', density=True, color='steelblue')
ax.hist(norms, bins=50, weights=result.weights, alpha=0.6, 
        label=f'Constrained (β={beta})', density=True, color='darkorange')

ax.set_xlabel('L2 Norm')
ax.set_ylabel('Density')
ax.set_title('Norm Distribution: Baseline vs Constrained')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

## 5. PCA Analysis

Examining how constraints affect the dimensionality structure.

In [ ]:
from sklearn.decomposition import PCA

# PCA on baseline
pca = PCA(n_components=min(20, VECTOR_DIM))
pca.fit(vectors)
baseline_var = np.cumsum(pca.explained_variance_ratio_)

# PCA on constrained (resampled)
rng = np.random.default_rng(42)
weights_norm = result.weights / result.weights.sum()
indices = rng.choice(N_SAMPLES, size=N_SAMPLES, replace=True, p=weights_norm)
resampled = vectors[indices]

pca_constrained = PCA(n_components=min(20, VECTOR_DIM))
pca_constrained.fit(resampled)
constrained_var = np.cumsum(pca_constrained.explained_variance_ratio_)

# Plot
fig, ax = plt.subplots(figsize=(8, 5))

components = np.arange(1, len(baseline_var) + 1)
ax.plot(components, baseline_var, 'o-', label='Baseline', color='steelblue', markersize=5)
ax.plot(components, constrained_var, 's-', label='Constrained', color='darkorange', markersize=5)
ax.axhline(0.9, color='gray', linestyle='--', alpha=0.5, label='90% threshold')

ax.set_xlabel('Number of Components')
ax.set_ylabel('Cumulative Explained Variance')
ax.set_title('PCA Explained Variance: Baseline vs Constrained')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 1.05)
plt.show()

In [ ]:
# 2D PCA scatter
pca_2d = PCA(n_components=2)
projected = pca_2d.fit_transform(vectors)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Baseline
n_show = min(2000, N_SAMPLES)
idx_baseline = rng.choice(N_SAMPLES, size=n_show, replace=False)
axes[0].scatter(projected[idx_baseline, 0], projected[idx_baseline, 1], 
                alpha=0.3, s=10, c='steelblue')
axes[0].set_xlabel('PC1')
axes[0].set_ylabel('PC2')
axes[0].set_title('Baseline')
axes[0].grid(True, alpha=0.3)

# Constrained
idx_constrained = rng.choice(N_SAMPLES, size=n_show, replace=True, p=weights_norm)
axes[1].scatter(projected[idx_constrained, 0], projected[idx_constrained, 1], 
                alpha=0.3, s=10, c='darkorange')
axes[1].set_xlabel('PC1')
axes[1].set_ylabel('PC2')
axes[1].set_title(f'Constrained (β={beta})')
axes[1].grid(True, alpha=0.3)

# Match axes
for ax in axes:
    ax.set_xlim(axes[0].get_xlim())
    ax.set_ylim(axes[0].get_ylim())

plt.suptitle('PCA Projection: Baseline vs Constrained', fontsize=12)
plt.tight_layout()
plt.show()

## 6. Matrix Experiments with Low-Rank Constraint

In [ ]:
# Initialize matrix constraints
lowrank = LowRankConstraint(target_rank=2, scale=1.0)
l1_matrix = L1Constraint(scale=0.05)

# Apply reweighting
matrix_constraints = [lowrank, l1_matrix]
matrix_result = apply_reweighting(matrices, matrix_constraints, beta=1.0, resample=True)

print(f"Matrix ESS: {matrix_result.ess:.1f} ({matrix_result.ess_ratio:.1%})")

In [ ]:
# Compute singular values
def get_singular_values(samples):
    svs = np.zeros((len(samples), samples.shape[1]))
    for i, m in enumerate(samples):
        svs[i] = np.linalg.svd(m, compute_uv=False)
    return svs

svs = get_singular_values(matrices)

# Baseline statistics
sv_mean_baseline = svs.mean(axis=0)
sv_std_baseline = svs.std(axis=0)

# Constrained statistics
w = matrix_result.weights / matrix_result.weights.sum()
sv_mean_constrained = (w[:, None] * svs).sum(axis=0)
sv_std_constrained = np.sqrt((w[:, None] * (svs - sv_mean_constrained)**2).sum(axis=0))

# Plot
fig, ax = plt.subplots(figsize=(8, 5))

indices = np.arange(1, MATRIX_SIZE + 1)
ax.errorbar(indices - 0.1, sv_mean_baseline, yerr=sv_std_baseline, 
            fmt='o-', label='Baseline', color='steelblue', capsize=3, markersize=6)
ax.errorbar(indices + 0.1, sv_mean_constrained, yerr=sv_std_constrained, 
            fmt='s-', label='Constrained', color='darkorange', capsize=3, markersize=6)

ax.axvline(2.5, color='gray', linestyle='--', alpha=0.5, label='Target rank boundary')

ax.set_xlabel('Singular Value Index')
ax.set_ylabel('Singular Value')
ax.set_title('Singular Value Spectrum: Baseline vs Constrained (target rank=2)')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xticks(indices)
plt.show()

## 7. Beta Sweep Analysis

Examining how constraint strength (beta) affects the ensemble.

In [ ]:
# Metrics across beta values
beta_range = np.linspace(0, 3, 20)
ess_values = []
mean_norms = []
mean_energies = []

for beta in beta_range:
    result = apply_reweighting(vectors, constraints, beta=beta)
    ess_values.append(result.ess_ratio)
    
    # Weighted mean norm
    norms = np.linalg.norm(vectors, axis=1)
    mean_norm = (result.weights * norms).sum()
    mean_norms.append(mean_norm)
    
    # Weighted mean energy
    mean_energy = (result.weights * result.energies).sum()
    mean_energies.append(mean_energy)

# Plot
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

axes[0].plot(beta_range, ess_values, 'o-', color='steelblue', markersize=4)
axes[0].set_xlabel('Beta')
axes[0].set_ylabel('ESS Ratio')
axes[0].set_title('Effective Sample Size vs Beta')
axes[0].grid(True, alpha=0.3)

axes[1].plot(beta_range, mean_norms, 'o-', color='darkorange', markersize=4)
axes[1].set_xlabel('Beta')
axes[1].set_ylabel('Mean Norm')
axes[1].set_title('Weighted Mean Norm vs Beta')
axes[1].grid(True, alpha=0.3)

axes[2].plot(beta_range, mean_energies, 'o-', color='green', markersize=4)
axes[2].set_xlabel('Beta')
axes[2].set_ylabel('Mean Energy')
axes[2].set_title('Weighted Mean Energy vs Beta')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

This demo illustrated the core CISE workflow:

1. **Baseline Generation**: Create ensembles from standard distributions
2. **Constraint Definition**: Define penalty functions as energy functions
3. **Reweighting**: Apply constraints via Boltzmann weights w(z) = exp(-beta * E(z))
4. **Analysis**: Compare baseline vs constrained distributions using various metrics

Key observations:
- Higher beta values lead to lower ESS (fewer effective samples)
- Constraints induce distributional shifts favoring low-energy configurations
- The low-rank constraint visibly suppresses smaller singular values

**Remember**: This is a constraint experiment exploring how penalty functions affect ensemble distributions. Results should not be interpreted as physical predictions.